# Skyfield Almanach
Voici les importations et les objets qui piloteront les exemples ci-dessous :

In [90]:
from skyfield import almanac
from skyfield.api import N, S, E, W, load, wgs84

ts = load.timescale()
eph = load('de421.bsp')
sun = eph['Sun']
cherbourg = wgs84.latlon(49.6386 * N, 1.6163 * W)
observer = eph['Earth'] + cherbourg

### Temps d’arrondi à la minute près

In [91]:
from datetime import datetime, timedelta
import pytz

def nearest_minute(dt):
    return (dt + timedelta(seconds=30)).replace(second=0, microsecond=0)

# Utiliser l'heure actuelle avec le fuseau horaire de Paris
paris_tz = pytz.timezone('Europe/Paris')

t = datetime.now(paris_tz)

dt = nearest_minute(t)
print(dt.strftime('%Y-%m-%d %H:%M'))



2024-08-30 23:54


Les résultats devraient alors concorder avec les tableaux produits par l’USNO et utilisés dans Skyfield

## Levers et couchers

### Lever et coucher du soleil

In [92]:
# Définir les temps de début et de fin pour le calcul
t0 = ts.utc(t.year, t.month, t.day)
t1 = ts.utc(t0.utc_datetime() + timedelta(days=1))

# Calculer les heures de lever et de coucher du soleil
f_sun = almanac.sunrise_sunset(eph, cherbourg)
times_sun, events_sun = almanac.find_discrete(t0, t1, f_sun)

for t, e in zip(times_sun, events_sun):
    rounded_time = nearest_minute(t.utc_datetime())
    rounded_time_paris = rounded_time.astimezone(paris_tz)
    print(f"{'Sunrise' if e else 'Sunset'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Sunrise: 2024-08-30 07:20
Sunset: 2024-08-30 20:53


In [93]:
# Calculer les heures de lever et de coucher de la lune
def moonrise_moonset(eph, location):
    t, y = almanac.find_discrete(t0, t1, almanac.risings_and_settings(eph, eph['Moon'], location))
    return t, y

times_moon, events_moon = moonrise_moonset(eph, cherbourg)

for t, e in zip(times_moon, events_moon):
    rounded_time = nearest_minute(t.utc_datetime())
    rounded_time_paris = rounded_time.astimezone(paris_tz)
    print(f"{'Moonrise' if e else 'Moonset'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Moonrise: 2024-08-30 02:46
Moonset: 2024-08-30 19:52


In [94]:
# Calculer les heures de lever et de coucher des planètes
planets = {
    'Mercury': 199,
    'Venus': 299,
    'Mars': 499,
    'Jupiter': 5,
    'Saturn': 6,
    'Uranus': 7,
    'Neptune': 8
}

for planet, code in planets.items():
    f_planet = almanac.risings_and_settings(eph, eph[code], cherbourg)
    times_planet, events_planet = almanac.find_discrete(t0, t1, f_planet)
    for t, e in zip(times_planet, events_planet):
        rounded_time = nearest_minute(t.utc_datetime())
        rounded_time_paris = rounded_time.astimezone(paris_tz)
        print(f"{planet} {'rise' if e else 'set'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Mercury rise: 2024-08-30 05:59
Mercury set: 2024-08-30 20:09
Venus rise: 2024-08-30 09:29
Venus set: 2024-08-30 21:41
Mars set: 2024-08-30 17:22
Mars rise: 2024-08-31 01:09
Jupiter set: 2024-08-30 16:41
Jupiter rise: 2024-08-31 00:40
Saturn set: 2024-08-30 08:14
Saturn rise: 2024-08-30 21:14
Uranus set: 2024-08-30 14:51
Uranus rise: 2024-08-30 23:27
Neptune set: 2024-08-30 09:25
Neptune rise: 2024-08-30 21:31
